Note: 
- Some checks already run for: wall visibility, flipped and rotate, loser's choice and trial proportion
- From conditioned player choice, loser inferred choice, opponent visibility, wall visibility and choice, get indices, trial list filters, 
    - retrievable choice
        - should always be retrievable for winner
        - should not be retrievable for loser if loser's choice conditions are not met
    - other visible
        - plot trials and check whether other visible output matches head angle orientation of player_id with respect to other at the start of the trial
    - one wall initially visible
        - plot trials with head angle trajectory and check that wall is initially visible or not comparing to function output
    - player chose given wall
        - loser's choice
            - plot losing player trials and check that choice (conditioned upon function logic) matches the trajectory
            - plot final third trajectory only
            - check cosine similarities between trajectory direction vector and player-alcove vectors for each wall
            - check that average most aligned trajectory output matches visual
            - check that highest alignment values match visually
            - check that final distance to most aligned wall matchest visually
            - print out output for session and compare to individual trial outputs
        - winner's choice 
            - check against json record of chosen wall index
    - opponent visibility
        - check angle to opponent against head angle trajectory plots
        --> specifically, plot head angle vector for player and vector to opponent for both players for every trial
    - wall visibility
        - wall first visible to check against head angle plots
        - wall visibility order against plots
        - wall1 and wall2 visibility against plots
        - first visible wall chosen session against plots for individual players for every trial within that session
    - get trials of trial type
        - compare len of output to number of trials of that given trial type in json for each trial type to confirm
    - was given wall chosen
        - print out whole output
    - filtering trials
        - get indices for filtered trials, plot and decide whether filtering worked correctly
        - get indices for filtered out trials, plot and decide whether exclusion is justified
    - retrievable choice
        - print 'get player wall choice' output
        - check that original indices match player choice indices where player choice is not nan
        - player chose wall filter against previously plotted trajectories
        - player wall visibility filters against previously plotted head angle trajectories

**OPPONENT VISIBILITY**

In [ ]:
# get access to the complete repo
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(),"..",".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path", main_copy_dir, octagon_analysis)

In [ ]:
# imports
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import analysis.opponent_visibility as opponent_visibility
import trajectory_analysis.trajectory_vectors as trajectory_vectors
import trajectory_analysis.trajectory_headangle as trajectory_headangle
import data_extraction.trial_list_filters as trial_list_filters
from plotting import plot_octagon, plot_trajectory
import utils.cosine_similarity as cosine_similarity
import data_extraction.extract_trial as extract_trial


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals




In [ ]:
# data folder and filenames

In [ ]:
# single json filename

In [ ]:
_, trial_list = prepare_data.prepare_data(data_folder, json_filename, combine=True)

In [ ]:
# global variables
player_id = 0
other_id = 1
trial_num = 10
trial = trial_list[trial_num]
fov = 110

In [ ]:
# gather data for head angle and vector to opponent

player_position_coordinates = opponent_visibility.get_player_position_slice_onset(player_id, trial)
other_position_coordinates = opponent_visibility.get_player_position_slice_onset(other_id, trial)

trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=player_id)
other_trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=other_id)

headangles = trajectory_vectors.extract_trial_player_headangles(trial=trial, player_id=player_id)
trial_player_headangles =  trajectory_headangle.get_player_headangle_vectors_for_trial(headangles)
trial_player_headangles_smoothed = trajectory_headangle.get_smoothed_player_head_angle_vectors_for_trial(headangles, window_size=5)

headangles_other = trajectory_vectors.extract_trial_player_headangles(trial=trial, player_id=other_id)
trial_player_headangles_other =  trajectory_headangle.get_player_headangle_vectors_for_trial(headangles)
trial_player_headangles_smoothed_other = trajectory_headangle.get_smoothed_player_head_angle_vectors_for_trial(headangles, window_size=5)

self_other_vector = other_position_coordinates - player_position_coordinates

# I don't think I'm using these
head_angle_vector = opponent_visibility.get_player_headangle_vector_slice_onset(player_id, trial)
head_angle_vector_other = opponent_visibility.get_player_headangle_vector_slice_onset(other_id, trial)

In [ ]:
# get output from other visibility functions

# compare to plotted thetas
# print out at the bottom in matrix form reflecting arrangement of plots
# each matrix (of 2) will contain angles between head angle vector and self to other vector for the two players
angle_to_opponent_player = opponent_visibility.get_angle_of_opponent_from_player_trial(player_id, trial)
angle_to_opponent_other = opponent_visibility.get_angle_of_opponent_from_player_trial(other_id, trial)

# compare to plotted vectors 
# print out at the bottom in matrix form reflecting arrangement of plots
# each matrix (of 2) will be a binary output for opponent visible (1) or not visible (0)
orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list)
orientation_angle_to_player_session = opponent_visibility.get_angle_of_opponent_from_player_session(other_id, trial_list)
other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, fov)
player_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_player_session, fov)

# trial list filters
(trial_list_filtered_other_visible,
 other_visible_trial_indices) = trial_list_filters.filter_trials_other_visible(trial_list, other_visible_session, inverse=inverse_other_visible)

In [1]:
# get visualisation vector coordinates
def get_other_visible_vectors(player_position_coordinates, other_position_coordinates, trial_player_headangles_smoothed, trial_player_headangles_smoothed_other, vector_length = 20, start_index = 0):
    ''' Return coordinates of vectors from player location to head angle direction 
    and from player to opponent'''

    # get player position coordinates at slice onset
    x_coordinate = player_position_coordinates[0,start_index]
    y_coordinate = player_position_coordinates[1,start_index]

    x_other_coordinate = other_position_coordinates[0,start_index]
    y_other_coordinate = other_position_coordinates[1,start_index]

    # get player head angle coordinates for smoothed head angles
    x_headangle = trial_player_headangles_smoothed[0,start_index]
    y_headangle = trial_player_headangles_smoothed[1,start_index]

    x_other_headangle = trial_player_headangles_smoothed_other[0,start_index]
    y_other_headangle = trial_player_headangles_smoothed_other[1,start_index]

    # get vector between players 
    player_position_flat = player_position_coordinates.flatten()
    other_position_flat = other_position_coordinates.flatten()

    self_other_vector = other_position_flat - player_position_flat
    other_self_vector = - self_other_vector
    self_other_vector_scaled = (self_other_vector/np.linalg.norm(self_other_vector)) * vector_length
    other_self_vector_scaled = - self_other_vector_scaled

    # assign start and end coordinates
    start = [x_coordinate, y_coordinate]
    other_start = [x_other_coordinate, y_other_coordinate]

    end = [x_coordinate + x_headangle * vector_length, y_coordinate + y_headangle * vector_length]
    other_end = [x_other_coordinate + x_other_headangle * vector_length, y_other_coordinate + y_other_headangle * vector_length]

    head_direction_vector_coordinates = np.array(list(zip(start,end)))
    head_direction_vector_coordinates_other = np.array(list(zip(other_start,other_end)))

    return(head_direction_vector_coordinates, head_direction_vector_coordinates_other,
           self_other_vector_scaled, other_self_vector_scaled,
           player_position_coordinates, other_position_coordinates)




#don't need the below just call get_angle_of_opponent_from_player_trial from opponent_visibility
def get_self_to_other_head_direction_angle(player_position, other_position, 
                                           trial_player_headangles_smoothed, trial_player_headangles_smoothed_other,
                                           start_index = 0):
    '''Cosine similarities between the vector of self head angle and vector from self to other'''
    
    x_headangle = trial_player_headangles_smoothed[0,start_index]
    y_headangle = trial_player_headangles_smoothed[1,start_index]

    x_other_headangle = trial_player_headangles_smoothed_other[0,start_index]
    y_other_headangle = trial_player_headangles_smoothed_other[1,start_index]

    player_headangle_vector = np.vstack([x_headangle, y_headangle])
    other_headangle_vector = np.vstack([x_other_headangle, y_other_headangle])

    player_theta = opponent_visibility.calculate_angle_to_opponent_from_positions_and_headangle(player_position, other_position, player_headangle_vector)
    other_theta = opponent_visibility.calculate_angle_to_opponent_from_positions_and_headangle(other_position, player_position, other_headangle_vector)

    return(player_theta, other_theta)





def get_other_visibility_testing_information(player_id, other_id, trial=None, trial_list=None, trial_num=None):
    '''Returns all data required to plot other visibility vectors, head angle vector, and angles
    for get_other_visible_vectors and plot_other_visibility_vectors'''

    trial = extract_trial.extract_trial(trial, trial_list, trial_num)

    player_position_coordinates = opponent_visibility.get_player_position_slice_onset(player_id, trial)
    other_position_coordinates = opponent_visibility.get_player_position_slice_onset(other_id, trial)

    trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=player_id)
    other_trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=other_id)

    headangles = trajectory_vectors.extract_trial_player_headangles(trial=trial, player_id=player_id)
    trial_player_headangles =  trajectory_headangle.get_player_headangle_vectors_for_trial(headangles)
    trial_player_headangles_smoothed = trajectory_headangle.get_smoothed_player_head_angle_vectors_for_trial(headangles, window_size=5)

    headangles_other = trajectory_vectors.extract_trial_player_headangles(trial=trial, player_id=other_id)
    trial_player_headangles_other =  trajectory_headangle.get_player_headangle_vectors_for_trial(headangles_other)
    trial_player_headangles_smoothed_other = trajectory_headangle.get_smoothed_player_head_angle_vectors_for_trial(headangles_other, window_size=5)

    #self_other_vector = other_position_coordinates - player_position_coordinates

    player_theta = opponent_visibility.get_angle_of_opponent_from_player_trial(player_id, trial)
    other_theta = opponent_visibility.get_angle_of_opponent_from_player_trial(other_id, trial)


    return(player_position_coordinates, other_position_coordinates,
           trajectory, other_trajectory, 
           trial_player_headangles, trial_player_headangles_other, 
           trial_player_headangles_smoothed, trial_player_headangles_smoothed_other,
           player_theta, other_theta)



def get_self_to_other_vectors_for_trajectory(trajectory, trajectory_other, player_id, other_id):
    '''Calculate the direction vector between one player and the other'''

    timepoints = trajectory.shape[1]
    timepoints_other = trajectory_other.shape[1]
    vectors_to_other = np.zeros((timepoints, 2))
    vectors_to_player = np.zeros((timepoints_other, 2))

    for time_index in range(0, trajectory.shape[1]):
        player_x_coordinate = trajectory[0,time_index]
        player_y_coordinate = trajectory[1,time_index]

        other_x_coordinate = trajectory_other[0,time_index]
        other_y_coordinate = trajectory_other[1,time_index]

        vector_to_other = trajectory_other[:,time_index] - trajectory[:,time_index]
        vector_to_player = trajectory[:,time_index] - trajectory_other[:,time_index]

        vectors_to_other[time_index,:] = vector_to_other
        vectors_to_player[time_index,:] = vector_to_player

    return(vectors_to_other, vectors_to_player)




def get_thetas_to_opponent_for_trial(vector_to_other, vector_to_player, trial_player_headangles, trial_player_headangles_other, trajectory):

    timepoints = trajectory.shape[1]
    thetas_player = np.zeros([1,timepoints])
    thetas_other = np.zeros([1,timepoints])

    for time_index in range(0, timepoints):

        dot_product_vectors_player = np.dot(vector_to_other[:,time_index].flatten(), trial_player_headangles[:,time_index].flatten())
        dot_product_vectors_other = np.dot(vector_to_player[:,time_index].flatten(), trial_player_headangles_other[:,time_index].flatten())

        (player_other_vector_norm,
         player_head_angle_vector_norm) = opponent_visibility.calculate_vector_norms_for_timepoint(vector_to_other[:,time_index],
                                                                                                   trial_player_headangles[:,time_index])
        
        (other_player_vector_norm,
         other_head_angle_vector_norm) = opponent_visibility.calculate_vector_norms_for_timepoint(vector_to_player[:,time_index],
                                                                                                   trial_player_headangles_other[:,time_index])
        
        vector_cosine_similarity_player = cosine_similarity.calculate_cosine_similarity_two_vectors(dot_product_vectors_player,
                                                                                                    player_other_vector_norm,
                                                                                                    player_head_angle_vector_norm)
        vector_cosine_similarity_other = cosine_similarity.calculate_cosine_similarity_two_vectors(dot_product_vectors_other,
                                                                                                    other_player_vector_norm,
                                                                                                    other_head_angle_vector_norm)

        thetas_player[:,time_index] = cosine_similarity.calculate_angle_from_cosine_similarity(vector_cosine_similarity_player)
        thetas_other[:,time_index] = cosine_similarity.calculate_angle_from_cosine_similarity(vector_cosine_similarity_other)

    return(thetas_player, thetas_other)




def plot_other_visibility_vectors(head_direction_vector_coordinates, head_direction_vector_coordinates_other,
                                  player_position_coordinates, other_position_coordinates, trajectory, trajectory_other,
                                  self_other_vector_scaled, other_self_vector_scaled,
                                  radius = 0.8, start_index = 0, colours = ['r', 'b', 'g','orange','purple'], axes=None):
    '''Return octagon plot with four vectors and two angles: head direction vector for player 0, 
    head direction vector for player 1, overlapping arrow vectors from one player to another,
    angles between player head direction vector and self-other vector'''

    # plot the octagon
    ax = plot_octagon.plot_octagon(ax=axes)

    # plot trajectories
    ax.scatter(trajectory[0, :], trajectory[1, :], s=0.5)
    ax.scatter(trajectory_other[0, :], trajectory_other[1, :], s=0.5)

    # NOTE: potentially add prior logic for plotting loser trajectory in blue and winner trajectory in red

    # plot head angle vectors
    ax.plot(head_direction_vector_coordinates[0,:], head_direction_vector_coordinates[1,:], c=colours[0], linewidth=2)
    ax.plot(head_direction_vector_coordinates_other[0,:], head_direction_vector_coordinates_other[1,:], c=colours[1], linewidth=2)

    # plot vector between players
    x_coordinate = player_position_coordinates[0,start_index]
    y_coordinate = player_position_coordinates[1,start_index]

    ax.plot(x_coordinate, y_coordinate, 'bo')

    x_other = other_position_coordinates[0,start_index]
    y_other = other_position_coordinates[1,start_index]

    ax.plot(x_other, y_other, 'bo')

    ax.quiver(x_coordinate, y_coordinate,
              self_other_vector_scaled[0], self_other_vector_scaled[1],
              angles='xy', scale_units='xy', scale=0.05, color=colours[3])

    ax.quiver(x_other, y_other,
              other_self_vector_scaled[0], other_self_vector_scaled[1],
              angles='xy', scale_units='xy', scale=0.05, color=colours[3])



    # plotting the angle between vectors
    angle1 = np.arctan2(*self_other_vector_scaled[::-1]) # np.arctan2 expects [y, x]
    angle2 = np.arctan2(*other_self_vector_scaled[::-1])

    # normalise agles to 0-2π 
    angle1 = angle1 % (2 * np.pi)
    angle2 = angle2 % (2 * np.pi)

    # compute difference 
    angle_diff = (angle1 - angle2) % (2 * np.pi)
    if angle_diff > np.pi:
        theta = 2 * np.pi - angle_diff
    else: 
        theta = angle_diff

    # pick smallest angle
    # if going the long way around, reverse direction
    if angle_diff > np.pi:
        # from angle 2 to angle 1
        arc_angles = np.linspace(angle2, angle1 + 2 * np.pi, 100)
    else:
        # from angle1 to angle2
        arc_angles = np.linspace(angle1, angle2, 100)

    # plot angle arc
    arc_x = x_coordinate + radius * np.cos(arc_angles)
    arc_y = y_coordinate + radius * np.sin(arc_angles)
    ax.plot(arc_x, arc_y, color='black')

    # label angle
    angle_label_x = x_coordinate + radius * np.cos((angle1 + angle2)/2)
    angle_label_y = y_coordinate + radius * np.sin((angle1 + angle2)/2)
    ax.text(angle_label_x, angle_label_y, f'{theta}°', fontsize=12, color='black')



def plot_single_trial_other_visibility(trial_list, trial_num, player_id, other_id):
    '''Single plot showing player head direction vectors, self to other vectors, angles between them and trajectories'''

    fig, ax = plt.subplots()

    # get all relevant information for trial other visibility
    (player_position_coordinates, other_position_coordinates,
    trajectory, other_trajectory, 
    trial_player_headangles, trial_player_headangles_other, 
    trial_player_headangles_smoothed, trial_player_headangles_smoothed_other,
    player_theta, other_theta) = get_other_visibility_testing_information(player_id, other_id, trial_list=trial_list, trial_num=trial_num)
    
    # get visualisation vector coordinates
    (head_direction_vector_coordinates, head_direction_vector_coordinates_other,
           self_other_vector_scaled, other_self_vector_scaled,
           player_position_coordinates, other_position_coordinates) = get_other_visible_vectors(player_position_coordinates, other_position_coordinates, 
                                                                                                trial_player_headangles_smoothed, trial_player_headangles_other, vector_length = 20, start_index = 0)
    
    # get angle between player head angle vector and vector to opponent for the entire trajectory
    (vector_to_other, vector_to_player) = get_self_to_other_vectors_for_trajectory(trajectory, other_trajectory, player_id, other_id)
    (thetas_player, thetas_other) = get_thetas_to_opponent_for_trial(vector_to_other, vector_to_player, trial_player_headangles, trial_player_headangles_other, trajectory)

    ax = plot_other_visibility_vectors(head_direction_vector_coordinates, head_direction_vector_coordinates_other,
                                        player_position_coordinates, other_position_coordinates, trajectory, other_trajectory,
                                        self_other_vector_scaled, other_self_vector_scaled,
                                        radius = 0.8, start_index = 0, colours = ['r', 'b', 'g','orange'], axes=ax)

    plt.show()
    
    thetas_player = round(thetas_player[:,:], 1)
    thetas_other = round(thetas_other[:,:], 1)

    return (thetas_player, thetas_other)



def plot_multiple_trials_other_visibility(rows, cols, player_id, other_id, trial_list, trial_num_offset=0, start_index=0):
    ''' Subplots showing player head direction vectors, self to other vectors, angles between them and trajectories'''

    fig, axes = plt.subplots(rows,cols,figsize=(20,20))

    player_thetas = np.full((rows,cols), np.nan, dtype=float)
    other_thetas = np.full((rows,cols), np.nan, dtype=float)

    for i in range(rows):
        for j in range(cols):
            trial_num = i*rows + j + trial_num_offset
            print(trial_num)

            try:
                # get all relevant information for trial other visibility
                (player_position_coordinates, other_position_coordinates,
                trajectory, other_trajectory, 
                trial_player_headangles, trial_player_headangles_other, 
                trial_player_headangles_smoothed, trial_player_headangles_smoothed_other,
                player_theta, other_theta) = get_other_visibility_testing_information(player_id, other_id, trial_list=trial_list, trial_num=trial_num)
                
                # get visualisation vector coordinates
                (head_direction_vector_coordinates, head_direction_vector_coordinates_other,
                    self_other_vector_scaled, other_self_vector_scaled,
                    player_position_coordinates, other_position_coordinates) = get_other_visible_vectors(player_position_coordinates, other_position_coordinates, 
                                                                                                trial_player_headangles_smoothed, trial_player_headangles_smoothed_other, vector_length = 20, start_index = 0)

                # get angle between player head angle vector and vector to opponent for the entire trajectory
                (vector_to_other, vector_to_player) = get_self_to_other_vectors_for_trajectory(trajectory, other_trajectory, player_id, other_id)
                # (thetas_player, thetas_other) = get_thetas_to_opponent_for_trial(vector_to_other, vector_to_player, trial_player_headangles, trial_player_headangles_other, trajectory)
            
            except Exception as e:
                axes[i,j].axis('off')
                continue

            axes[i,j] = plot_other_visibility_vectors(head_direction_vector_coordinates, head_direction_vector_coordinates_other,
                                        player_position_coordinates, other_position_coordinates, trajectory, other_trajectory,
                                        self_other_vector_scaled, other_self_vector_scaled,
                                        radius = 0.8, start_index = 0, colours = ['r', 'b', 'g','orange'], axes=axes[i,j])
            
            #player_theta[i,j] = round(player_theta[:,start_index], 1)
            #other_theta[i,j] = round(other_theta[:,start_index], 1)

    plt.tight_layout()

    plt.show()

    #return (player_theta, other_theta)
